# Scraping High Charts on Website

We have to verify the data showing on a website using high charts. We already checked the data on the database that feeds this chart. But, since there are some calculations done on the API call, we also would like to verify the actual chart to make sure there are no errors on the call.

In order to do this, we'll call the api from our python code and store the results, export into a csv file to compare in another script with our actual data

In [ ]:
#let's begin by testing the api call
import requests
myUrl = '####'
parameters = {'##':'#####','###':'##'}
headers = {'##': '##/###, ##/####, */*',
        '####': '##, ###, ####',
        '####-####': '####'}
r = requests.get(url=myUrl,params = parameters, headers=headers)
r_data = r.json()

In [ ]:
size = len(r_data[0]['graphData'])
results = {}
for i in range(0,size):
    results[r_data[0]['graphData'][i]['name']]=r_data[0]['graphData'][i]['total']['value']


In [ ]:
print(results)

We can see our api call is working. But, we want to store the results for all our locations. So, let's create a dictionary with our locations codes and names, iterate over it, call the api on each element, store the results on our results dictionary based on the location name

In [ ]:
#let's rewrite our code above to store location name and results
size = len(r_data[0]['graphData'])
location_results = {}
results = {}
for i in range(0,size):
    location_results[r_data[0]['graphData'][i]['name']]=r_data[0]['graphData'][i]['total']['value']
results[parameters.get('###')]=location_results
print(results)

In [ ]:
#we're now ready to bring in the locations data and iterate over it doing our api call and store the results
#let's bring in the locations name from an excel file

import pandas as pd
campuses = pd.read_excel('####.xlsx',sheet_name='####',header=None,names=['####','####'])
campuses

In [ ]:
campus_dict = campuses.set_index('id')['campus'].to_dict()
print(campus_dict)

In [ ]:
#we now have a dictionary with the location ids and the location names. We can now iterate over it and do the api call for each
results = {}
for key, value in campus_dict.items():
    parameters = {'type':'YearToDateAverages','####':key}
    r = requests.get(url=myUrl,params = parameters, headers=headers)
    r_data = r.json()
    size = len(r_data[0]['graphData'])
    location_results = {}
    for i in range(0,size):
        location_results[r_data[0]['graphData'][i]['name']]=r_data[0]['graphData'][i]['total']['value']
    results[value]=location_results
print(results)

In [ ]:
api_df = pd.DataFrame.from_dict(results,orient='index',columns=['campus','2019','2018','2017'])

In [ ]:
api_df['campus']=api_df.index
api_df = api_df.reset_index()
api_df = api_df.drop(columns=['index'])
api_df

In [ ]:
#writing results to xlsx file for further analysis
writer = pd.ExcelWriter('api_class101.xlsx',engine='xlsxwriter')
api_df.to_excel(writer,sheet_name='class101')
writer.save()